# Notebook for preprocessing SPED data

This notebook contains most of the preprocessing steps required for phasemapping in SPED data using either vector matching, neural networks, machine learning, or template matching. It should work with pyxem version >=0.14 (dev) and hyperspy >=0.7.0 (dev).

The preprocessing steps are as follows:
 - Cropping
 - Centering by fitting a linear descan plane based on center of mass  (COM)
 - Calibration by averaging estimated scales in two different directions
 - Rebinning
 - Preparation of masks to be stored in metadata
 
The results/steps from the centering and the masks are stored in the metadata field `signal.metadata.Preprocessing`.

The only step not performed is normalization. The reason for this is that this step requires the dataset to be converted to `float32`, which is about four times as large as the `uint16` or two times as large as the `uint32` raw datasets. Hence, methods that require normalization should peform those locally in order to reduce storage requirements and file transfer rates and times.

In [1]:
%matplotlib
%config IPCompleter.use_jedi=False
import hyperspy.api as hs
import pyxem as pxm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
from pathlib import Path
from skimage.feature import blob_dog, blob_log, blob_doh

Using matplotlib backend: Qt5Agg


# Loading raw data

In [2]:
data_path = Path(r'2021_10_06_2xxx_24h_250C/Site2/SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3.hspy')
signal = hs.load(data_path, lazy=True)

/home/emilc/miniconda3/envs/pyxem-dev/lib/python3.10/site-packages/hyperspy/misc/utils.py:468: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
/home/emilc/miniconda3/envs/pyxem-dev/lib/python3.10/site-packages/hyperspy/io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


## Crop the data

In [3]:
signal = signal.inav[0:512, 0:512]

## Centering

Plot the maximum to get an idea of all the reflections through the dataset, including the travel of the direct beam. Adjust the ROI to include __only__ the direct beam (or at least not too many other reflections) - this will be used for calculating the COM that will be used when fitting a linear descan slope/plane later on

In [6]:
maximums = signal.max(axis=[0, 1])
maximums.compute()

[########################################] | 100% Completed |  2min 23.2s


In [7]:
maximums.plot(norm='symlog')
roi = hs.roi.CircleROI(0, 0, 0.05)
roi.add_widget(maximums)

Convert the ROI parameters to unitless coordinates

In [11]:
x = (roi.cx - maximums.axes_manager[0].offset) / maximums.axes_manager[0].scale
y = (roi.cy - maximums.axes_manager[1].offset) / maximums.axes_manager[1].scale
r = roi.r / maximums.axes_manager[0].scale
print(f'I will look for COM within {r} pixels from ({x}, {y})')

I will look for COM within 12.50000001 pixels from (127.0, 125.99999999999999)


Calculate the Centre of mass (COM) within the region defined by the ROI

In [12]:
com = signal.center_of_mass(mask=(x, y, r))
com.plot()
com.get_bivariate_histogram().plot()

[########################################] | 100% Completed |  4min 46.6s


Calculate the beam shift by estimating a linear plane

In [38]:
beam_shift = pxm.signals.BeamShift(com.T)
mask = hs.signals.Signal2D(np.zeros(signal.axes_manager.navigation_shape, dtype=bool).T).T
mask.inav[20:-20, 20:-20] = True
beam_shift.make_linear_plane(mask=mask)
beam_shift = beam_shift - (signal.axes_manager.signal_shape[0]//2) #shift the beam to the center of the diffraction patterns
beam_shift.plot()

Shift the diffraction patterns

In [39]:
shifted_s = signal.shift_diffraction(beam_shift.isig[0], beam_shift.isig[1])

Update the metadata

In [49]:
shifted_s.metadata.add_dictionary({'Preprocessing':{'Centering': {'COM': {'COM': com, 'Mask': {'x': x, 'y': y, 'r':r}}, 'shifts': beam_shift, 'shift_estimate_mask': mask}}})
shifted_s.metadata

Make a smaller signal (taking every 8th pixel) to inspect the centering

In [42]:
s = shifted_s.inav[0::8, 0::8]
s.compute()

Plot the small signal

In [45]:
s.plot(norm='symlog')

## Calibration

Plot the signal, and navigate to a pixel with an easily identifiable diffraction pattern

In [16]:
signal.plot(norm='symlog')

[########################################] | 100% Completed |  2.7s


Plot the image at the current position and add two ROIs. Adjust the ROIs to measure the __same__ g-spacing but in different directions.

In [17]:
image = signal.inav[signal.axes_manager.indices]
image.plot(norm='symlog')
lines = [hs.roi.Line2DROI(*signal.axes_manager.signal_extent) for i in range(2)]
[line.add_widget(image) for line in lines]

Create profiles along the lines adjusted in the previous step, plot them, and add span ROIs to the profile plots. Adjust the span edges to lie in the middle of the g-spacings to measure

In [18]:
profiles = [line(image) for line in lines]
[profile.plot(norm='log') for profile in profiles]
spans = [hs.roi.SpanROI(0, 10) for i in range(2)]
[span.add_widget(profile) for (span, profile) in zip(spans, profiles)]

Calculate the scale for the different directions. Update the g-spacing details to suit your material.

In [38]:
a = 4.05
hkl = np.array([8, 0, 0])
g = np.sqrt(np.sum((hkl/a)**2))

scales = [g / ((span.right - span.left)/profile.axes_manager[0].scale) for (span, profile) in zip(spans, profiles)]
print(f'Calculated scales for directions: {scales} {image.axes_manager[0].units}')
print(f'Average scale: {np.mean(scales):.6f} {image.axes_manager[0].units}')
print(f'Standard deviation: {np.std(scales):.6f} {image.axes_manager[0].units}')

Calculated scales for directions: [0.009542553825967675, 0.009496676163342831] $A^{-1}$
Average scale: 0.009520 $A^{-1}$
Standard deviation: 0.000023 $A^{-1}$


Set the scale to be the average of the two scales.

In [39]:
scale = np.mean(scales)
shifted_s.set_diffraction_calibration(scale)

Check calibration

In [40]:
image = shifted_s.inav[signal.axes_manager.indices]

In [45]:
image.plot(norm='symlog', cmap='inferno')
g = np.sqrt(np.sum((np.array([2, 2, 0]) / 4.05)**2))
roi = hs.roi.CircleROI(0, 0, g)
roi.add_widget(image)

## Binning

Bin the shifted signal

In [46]:
binned_shifted_s = shifted_s.rebin(scale=(1, 1, 2, 2))

## Masking direct beam and reflections

Calculate the maximums

In [47]:
maximums = binned_shifted_s.max(axis=[0, 1])
maximums.compute()

[###                                     ] | 8% Completed | 19.0s


KeyboardInterrupt: 

Plot the data and navigate to a pure phase you want to mask out

In [48]:
binned_shifted_s.plot(norm='symlog')

[##                                      ] | 7% Completed | 14.8s


KeyboardInterrupt: 

Extract the current diffraction pattern

In [98]:
image = binned_shifted_s.inav[binned_shifted_s.axes_manager.indices]
image.plot(norm='symlog')

Find blobs in the current diffraction pattern and compare the mask to the maximum through stack image. Adjust `threshold` and `minimum_r` to get satisfactory masks for the direct beam and the other masks.

In [226]:
minimum_r = 5
threshold = 1E-18

blobs = blob_log(image.data, min_sigma=1, max_sigma=15, num_sigma=100, overlap=0, threshold=threshold)
nx, ny = maximums.axes_manager.signal_shape
mask = np.zeros((nx, ny), dtype=bool)
direct_beam_mask = np.zeros((nx, ny), dtype=bool)
xs = np.arange(0, nx)
ys = np.arange(0, ny)
X, Y = np.meshgrid(xs, ys)
for blob in blobs:
    y, x, r = blob
    r = np.sqrt(2)*r
    r = max([minimum_r, r])
    print(f'Adding mask with radius {r} at ({x}, {y})')
    R = np.sqrt((X-x)**2 + (Y-y)**2)
    if nx//2-2<=x<=nx//2+2 and ny//2-2<=y<=ny//2+2:
        direct_beam_mask[R<=r] = True
    else:
        mask[R<=r] = True
        
direct_beam_mask = ~direct_beam_mask
mask = ~mask
if len(blobs) >0:
    fig, ax = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True)
    ax[0, 0].imshow(mask, cmap='RdBu')
    ax[1, 0].imshow(direct_beam_mask, cmap='RdBu')
    ax[0, 1].imshow(mask*maximums.data, norm=SymLogNorm(0.03))
    ax[1, 1].imshow(direct_beam_mask*maximums.data, norm=SymLogNorm(0.03))

Adding mask with radius 5 at (64.0, 64.0)
Adding mask with radius 5 at (51.0, 41.0)
Adding mask with radius 5 at (86.0, 51.0)
Adding mask with radius 5 at (76.0, 86.0)
Adding mask with radius 5 at (41.0, 77.0)
Adding mask with radius 5 at (74.0, 28.0)
Adding mask with radius 5 at (99.0, 73.0)
Adding mask with radius 5 at (29.0, 54.0)
Adding mask with radius 5 at (54.0, 99.0)
Adding mask with radius 5 at (6.0, 67.0)
Adding mask with radius 5 at (67.0, 121.0)
Adding mask with radius 5 at (32.0, 112.0)
Adding mask with radius 5 at (89.0, 109.0)
Adding mask with radius 5 at (96.0, 16.0)
Adding mask with radius 5 at (19.0, 90.0)
Adding mask with radius 5 at (16.0, 32.0)
Adding mask with radius 5 at (61.0, 7.0)
Adding mask with radius 5 at (109.0, 38.0)
Adding mask with radius 5 at (39.0, 19.0)
Adding mask with radius 5 at (111.0, 96.0)
Adding mask with radius 5 at (9.0, 127.0)
Adding mask with radius 5 at (122.0, 60.0)
Adding mask with radius 5 at (5.0, 10.0)


In [51]:
binned_shifted_s.metadata.add_dictionary({'Preprocessing': {'Masks': {'Diffraction': {'direct_beam': direct_beam_mask, 'reflections': mask}}}})

## Masking navigation space

Mask out bad or uninteresting areas in navigation space. Run parts of this code several times to add more areas to the mask

Define empty navigation mask

In [210]:
navigation_mask = hs.signals.Signal2D(np.zeros(binned_shifted_s.axes_manager.navigation_shape, dtype=bool)).T
for ax in range(2):
    navigation_mask.axes_manager[ax].scale = binned_shifted_s.axes_manager[ax].scale
    navigation_mask.axes_manager[ax].name = binned_shifted_s.axes_manager[ax].name
    navigation_mask.axes_manager[ax].units = binned_shifted_s.axes_manager[ax].units
    navigation_mask.axes_manager[ax].offset = binned_shifted_s.axes_manager[ax].offset   

Create simple VBF for visualisation

In [174]:
nx, ny = binned_shifted_s.axes_manager.signal_shape
vbf = binned_shifted_s.isig[nx//2-16:nx//2+16, ny//2-16:ny//2+16].sum(axis=[-1, -2]).T
vbf.compute()

[########################################] | 100% Completed | 24.3s


Plot VBF and adda rectangular widget

In [179]:
vbf.plot()
roi = hs.roi.RectangularROI(*[binned_shifted_s.axes_manager.navigation_extent[i]/2 for i in [0, 2, 1, 3]])
roi.add_widget(vbf)

Move widget to a part you want to mask out, and adjust its size. Run the next cell to add the ROI to the mask

In [223]:
navigation_mask.inav[roi.left:roi.right, roi.top:roi.bottom] = True
navigation_mask.plot()

Add the navigation mask to the metadata

In [53]:
binned_shifted_s.metadata.add_dictionary({'Preprocessing': {'Masks': {'Scan': navigation_mask.data}}})

# Save the preprocessed data

In [ ]:
binned_shifted_s.save(data_path.with_name(f'{data_path.stem}_preprocessed_new.hspy'), overwrite=True, chunks=(32, 32, 32, 32))

# Load the preprocessed data for inspection and verification

In [5]:
preprocessed_signal = hs.load(data_path.with_name(f'{data_path.stem}_preprocessed.hspy'), lazy=True)